In [1]:
import torch
import torch.nn as nn
from typing_extensions import Final
from typing import Any, Tuple

In [2]:
class Model(nn.Module):
    def forward(self, x):
        N, C, T, H, W = x.shape
        value = x.view(N, 1, C, T * H * W)
        return value
    
model = Model()
data1 = torch.rand(2, 4, 8, 7, 7)
data2 = torch.rand(2, 4, 8, 10, 10)

In [11]:
traced_model = torch.jit.trace(model, [data1])
print(traced_model(data1).shape)
print(traced_model(data2).shape)

torch.Size([2, 1, 4, 392])
torch.Size([2, 1, 4, 800])


In [4]:
def calibrate(model, data_loader):
    with torch.no_grad():
        for data in data_loader:
            model(data)

In [6]:
qmodel = torch.quantization.quantize_jit(
    traced_model,
    {"": torch.quantization.default_qconfig},
    calibrate,
    [[data1]]
)

print(qmodel(data1).shape)
print(qmodel(data2).shape)

torch.Size([2, 1, 4, 392])


/home/zafar/Git/pytorch-dev/pytorch/torch/quantization/observer.py:115: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
<ipython-input-2-35c37cdc7bdc>(4): forward
/home/zafar/Git/pytorch-dev/pytorch/torch/nn/modules/module.py(709): _slow_forward
/home/zafar/Git/pytorch-dev/pytorch/torch/nn/modules/module.py(725): _call_impl
/home/zafar/Git/pytorch-dev/pytorch/torch/jit/_trace.py(953): trace_module
/home/zafar/Git/pytorch-dev/pytorch/torch/jit/_trace.py(755): trace
<ipython-input-3-273da0aec9fd>(1): <module>
/home/zafar/miniconda3/envs/pytorch-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3343): run_code
/home/zafar/miniconda3/envs/pytorch-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3263): run_ast_nodes
/home/zafar/miniconda3/envs/pytorch-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3072): run_cell_async
/home/zafar/miniconda3/envs/pytorch-dev/lib/python3.6/site-packages/IPython/core/async_helpers.py(68): _pseudo_sync_ru

In [12]:
traced_model.graph

graph(%self : __torch__.___torch_mangle_6.Model,
      %x : Float(2:1568, 4:392, 8:49, 7:7, 7:1, requires_grad=0, device=cpu)):
  %3 : int = prim::Constant[value=0]() # <ipython-input-2-35c37cdc7bdc>:3:0
  %4 : int = aten::size(%x, %3) # <ipython-input-2-35c37cdc7bdc>:3:0
  %N : Long(device=cpu) = prim::NumToTensor(%4)
  %20 : int = aten::Int(%N)
  %6 : int = prim::Constant[value=1]() # <ipython-input-2-35c37cdc7bdc>:3:0
  %7 : int = aten::size(%x, %6) # <ipython-input-2-35c37cdc7bdc>:3:0
  %C : Long(device=cpu) = prim::NumToTensor(%7)
  %21 : int = aten::Int(%C)
  %9 : int = prim::Constant[value=2]() # <ipython-input-2-35c37cdc7bdc>:3:0
  %10 : int = aten::size(%x, %9) # <ipython-input-2-35c37cdc7bdc>:3:0
  %T : Long(device=cpu) = prim::NumToTensor(%10)
  %12 : int = prim::Constant[value=3]() # <ipython-input-2-35c37cdc7bdc>:3:0
  %13 : int = aten::size(%x, %12) # <ipython-input-2-35c37cdc7bdc>:3:0
  %H : Long(device=cpu) = prim::NumToTensor(%13)
  %15 : int = prim::Constant[value=4](

In [10]:
qmodel.graph

graph(%self : __torch__.___torch_mangle_5.Model,
      %x : Float(2:1568, 4:392, 8:49, 7:7, 7:1, requires_grad=0, device=cpu)):
  %36 : int[] = prim::Constant[value=[2, 1, 4, 392]]()
  %24 : Float(2:1568, 1:1568, 4:392, 392:1, requires_grad=0, device=cpu) = aten::view(%x, %36) # <ipython-input-2-35c37cdc7bdc>:4:0
  return (%24)